<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - KNN Item Item MRR</h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [1]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, item_knn as knn
from lenskit import topn
import pandas as pd
import numpy as np

ratings = pd.read_csv('C:\\Users\\Alejo\\Tesis\\Demo\\ml-100k\\u.data', sep='\t',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # Entrenamos el algoritmo con el training dataset
    users = test.user.unique() # Devuelve un array con los users unicos en el testing dataset
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [7]:
def model_trainer(neighbours,center):
    neighbours=int(neighbours)
    all_recs = []
    test_data = []
    algo_ii = knn.ItemItem(neighbours,center=center,aggregate='weighted-average')
    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test)
        all_recs.append(eval('ItemItem', algo_ii, train, test))
    all_recs = pd.concat(all_recs, ignore_index=True)
    test_data = pd.concat(test_data, ignore_index=True)
    rla = topn.RecListAnalysis()
    rla.add_metric(topn.recip_rank)
    results = rla.compute(all_recs, test_data)
    return results["recip_rank"].mean()

In [9]:
from hyperopt import fmin, tpe, hp, STATUS_OK

def objective(params):
    neighbours = params['neighbours']
    center = params['center']
    metric = model_trainer(neighbours,center)
    print([neighbours,center,metric],',')
    return {'loss': -metric, 'status': STATUS_OK }

space= {'neighbours': hp.uniform('neighbours', 1, 100),
       'center': hp.choice('center', ['True', 'False'])}

best = fmin(objective, space, algo=tpe.suggest,max_evals=100)

[15.419260053750078, 'False', 0.12003451303838955]                                                                     
,                                                                                                                      
[11.242055065827829, 'True', 0.10623051259664701]                                                                      
,                                                                                                                      
[29.259429954165117, 'True', 0.09664575898435132]                                                                      
,                                                                                                                      
[52.28116879756481, 'True', 0.08607795115359618]                                                                       
,                                                                                                                      
[35.25510486107176, 'False', 0.095451724

[29.751064061918367, 'False', 0.10071774035798514]                                                                     
,                                                                                                                      
[21.54632624448368, 'False', 0.11087883894786187]                                                                      
,                                                                                                                      
[63.767014608806335, 'True', 0.08103666512216721]                                                                      
,                                                                                                                      
[13.33504386936037, 'False', 0.1156194793145262]                                                                       
,                                                                                                                      
[4.756111518473384, 'False', 0.114648345

[18.801571091694115, 'False', 0.11185727959567701]                                                                     
,                                                                                                                      
[38.11205722121494, 'False', 0.09159699257094063]                                                                      
,                                                                                                                      
[4.932857415498628, 'False', 0.11247593207570643]                                                                      
,                                                                                                                      
[30.097146162360765, 'False', 0.09522345626160256]                                                                     
,                                                                                                                      
[27.404603524904147, 'False', 0.10181884

In [11]:
metric = pd.DataFrame(array)
metric.columns = ['neighbours','center','MRR']
metric

,neighbours,center,MRR
0,15.419260,False,0.120035
1,11.242055,True,0.106231
2,29.259430,True,0.096646
3,52.281169,True,0.086078
4,35.255105,False,0.095452
...,...,...,...
95,17.638855,False,0.105849
96,67.422810,True,0.082109
97,9.182720,False,0.110587
98,35.631710,False,0.094204


In [13]:
metric.sort_values(by=['MRR'],ascending=False).head(10)

,neighbours,center,MRR
64,7.089107,False,0.121991
40,16.000415,False,0.121402
42,16.652362,False,0.120633
93,13.237834,False,0.120179
33,7.782480,False,0.120137
0,15.419260,False,0.120035
75,11.121970,False,0.119454
66,11.484438,False,0.119179
74,16.131556,False,0.118977
27,9.406124,False,0.118860


In [14]:
metric.to_csv('metric_MRR.csv', index=False)